In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install captum
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=79bee5658ccf704115eb28bcc2d93e08861837bb58e3658167ab0eaf68465439
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=a15931a33f9d80f68ecdd0d41ad0a2df714dc39d119af2b394c5024ee716b091
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from captum.attr import visualization as viz
from fvcore.nn import FlopCountAnalysis, flop_count_table
from captum.attr import IntegratedGradients, LayerConductance, DeepLift, LayerDeepLift,LayerIntegratedGradients
import itertools
import pandas as pd
import os
import random
import numpy as np
from itertools import product
from torch.utils.data import SequentialSampler, RandomSampler
from torch.utils.data import DataLoader

In [4]:
method_names = ["LayerIntegratedGradients", "LayerDeepLift"]
INPUT_SHAPE= (1, 3, 32, 32)
KERNEL_SIZES = [5, 5, 5]
PADDING = [2, 2, 2]
Training_Device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Feature_Attribution_Device = torch.device("cpu")
NUM_EPOCHS=3
NUM_CLASSES=10
IN_CHANNELS=3
NUM_RUN=15

dataset_name = "Baseline SVHN"
folder_path = '/content/drive/My Drive/2023 InterpretingNN/code/experiment and result stage3/' + dataset_name
filename = '/Baseline SVHN 5*5_result.csv'
file_path = folder_path+filename
print(file_path)

/content/drive/My Drive/2023 InterpretingNN/code/experiment and result stage3/Baseline SVHN/Baseline SVHN 5*5_result.csv


# Model structure defination

In [5]:
class Inception(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(Inception, self).__init__()
        stride = 1  # Common stride for all branches

        # Define branches with different kernel sizes and paddings
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=KERNEL_SIZES[0], stride=stride, padding=PADDING[0]),
            nn.ReLU()
        )
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=KERNEL_SIZES[1], stride=stride, padding=PADDING[1]),
            nn.ReLU()
        )
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=KERNEL_SIZES[2], stride=stride, padding=PADDING[2]),
            nn.ReLU()
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=stride, padding=1),
        )


        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(27648, num_classes),
        )

    def forward(self, x):
        x.requires_grad_(True)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)
        x = torch.cat((x1, x2, x3, x4), dim=1)  # Concatenating along the channel dimension
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layer
        x = self.fc(x)
        return x


# Get dataset

In [6]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.SVHN('.', split='train', download=True, transform=transform)
test_dataset = datasets.SVHN('.', split='test', download=True, transform=transform)

100%|██████████| 182040794/182040794 [00:13<00:00, 13031064.12it/s]


100%|██████████| 64275384/64275384 [00:05<00:00, 11349654.04it/s]


# FLOP Count

In [7]:
def count_flops(model, input_size):
    inputs = torch.randn(input_size)
    flops = FlopCountAnalysis(model, inputs)
    return flop_count_table(flops)

inception_model = Inception(IN_CHANNELS, NUM_CLASSES)
# Assuming the input size for CIFAR-10 (batch size, channels, height, width)
flops_table = count_flops(inception_model, INPUT_SHAPE)
print(flops_table)

| module             | #parameters or shape   | #flops   |
|:-------------------|:-----------------------|:---------|
| model              | 0.278M                 | 2.12M    |
|  branch1.0         |  0.608K                |  0.614M  |
|   branch1.0.weight |   (8, 3, 5, 5)         |          |
|   branch1.0.bias   |   (8,)                 |          |
|  branch2.0         |  0.608K                |  0.614M  |
|   branch2.0.weight |   (8, 3, 5, 5)         |          |
|   branch2.0.bias   |   (8,)                 |          |
|  branch3.0         |  0.608K                |  0.614M  |
|   branch3.0.weight |   (8, 3, 5, 5)         |          |
|   branch3.0.bias   |   (8,)                 |          |
|  fc.0              |  0.276M                |  0.276M  |
|   fc.0.weight      |   (10, 27648)          |          |
|   fc.0.bias        |   (10,)                |          |


# Train and attribution functions

train and eval function

In [8]:
def train(epoch, model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

    train_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Train Loss = {train_loss / len(train_loader):.4f}, Train Accuracy = {train_accuracy:.2f}%")

    return train_accuracy

def test(epoch, model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    test_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Test Loss = {test_loss / len(test_loader):.4f}, Test Accuracy = {test_accuracy:.2f}%")

    return test_accuracy


functions for calculate attribution

In [9]:
def print_ig(test_loader, model, device, num_batches=5):
    # Move the model to the device (CPU or CUDA)
    model.to(device)

    # Ensure the model is in evaluation mode
    model.eval()

    # Dictionary to store cumulative attributions for each layer
    cumul_ig_attributions = {}

    # Process 'num_batches' batches
    for _ in range(num_batches):
        inputs, target_class = next(iter(test_loader))
        inputs = inputs.to(device)

        # Iterate through each named module and compute attributions for Conv2d layers with learnable parameters
        for layer_name, layer_module in model.named_modules():
            # Check if the layer is a Conv2d layer with learnable parameters
            if isinstance(layer_module, nn.Conv2d) or isinstance(layer_module, nn.MaxPool2d):
                # Initialize LayerIntegratedGradients for the layer
                lig = LayerIntegratedGradients(model, layer_module)

                # Compute the attributions for the current layer
                try:
                    attributions = lig.attribute(inputs, target=target_class.to(device))
                except Exception as e:
                    print(f"Error computing attributions for layer {layer_name}: {e}")
                    continue

                # Sum up the attributions for the current layer
                attr_sum = attributions.cpu().detach().numpy().sum()

                # Add to the cumulative sum for the layer
                if layer_name in cumul_ig_attributions:
                    cumul_ig_attributions[layer_name] += attr_sum
                else:
                    cumul_ig_attributions[layer_name] = attr_sum

                # Free up memory
                del attributions, lig

    # Calculate and print the average attributions for each layer
    avg_ig_attributions = {layer: total_sum / num_batches for layer, total_sum in cumul_ig_attributions.items()}
    for layer_name, avg_attr in avg_ig_attributions.items():
        print(f'Layer: {layer_name}')
        print(f'Average Attribution: {avg_attr}')

    return avg_ig_attributions


def print_deeplift(test_loader, model, device, num_batches=5):
    # Move the model to the specified device and set it to evaluation mode
    model.to(device).eval()

    # Dictionary to store cumulative DeepLIFT attributions for each layer
    cumul_dl_attributions = {}

    # Process 'num_batches' batches
    for _ in range(num_batches):
        inputs, target_class = next(iter(test_loader))
        inputs, target_class = inputs.to(device), target_class.to(device)

        # Compute the attributions for Conv2d layers
        for layer_name, layer_module in model.named_modules():
            # Skip the whole model's container and focus on Conv2d layers with learnable parameters
            if isinstance(layer_module, nn.Conv2d) or isinstance(layer_module, nn.MaxPool2d):
                # Initialize LayerDeepLift with the current layer
                ldl = LayerDeepLift(model, layer_module)

                # Compute the attributions for the current layer
                attributions_ldl = ldl.attribute(inputs, target=target_class.to(device))

                # Sum up the attributions for the current layer
                attr_sum = attributions_ldl.cpu().data.numpy().sum()

                # Add to the cumulative sum for the layer
                if layer_name in cumul_dl_attributions:
                    cumul_dl_attributions[layer_name] += attr_sum
                else:
                    cumul_dl_attributions[layer_name] = attr_sum

                del attributions_ldl, ldl

    # Calculate and print the average attributions for each layer
    avg_dl_attributions = {layer: total_sum / num_batches for layer, total_sum in cumul_dl_attributions.items()}
    for layer_name, avg_attr in avg_dl_attributions.items():
        print(f'Layer: {layer_name}')
        print(f'Average Attribution: {avg_attr}')

    return avg_dl_attributions

# Possible Hyperparameter grid search creation

in hyperparams_list_dict, each hyperparameter has corresponding possible choices as a list, during experiment, given hyperparams sequence, location each hyperperameter's location using hyperparameter encoding function to convert strings or classes back into their index in hyperparams_list_dict

In [10]:
'''sample_hyperparams_list_dict = {
    'initial_lr': [],
    'optimizer': torch.optim.Adam,  # Example optimizer
    'criterion': torch.nn.CrossEntropyLoss(),
    'train_data_used': 0.8,
    'train_set_shuffle': True,
    'train_batch_size': 64
}'''

"sample_hyperparams_list_dict = {\n    'initial_lr': [],\n    'optimizer': torch.optim.Adam,  # Example optimizer\n    'criterion': torch.nn.CrossEntropyLoss(),\n    'train_data_used': 0.8,\n    'train_set_shuffle': True,\n    'train_batch_size': 64\n}"

In [11]:
def generate_hyperparameter_combinations(hyperparams):
    """
    Generate a sequence of hyperparameter combinations.

    :param hyperparams: A dictionary where keys are the names of hyperparameters,
                        and values are lists of possible choices for each hyperparameter.
    :return: A list of dictionaries, each representing a unique combination of hyperparameters.
    """
    # Extract the hyperparameter names and their corresponding choices
    keys, values = zip(*hyperparams.items())

    # Generate all possible combinations of hyperparameter values
    all_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    return all_combinations

# Automated Experiments

In [12]:
def get_data_loader(hyperparams, train_dataset, test_dataset):
  shuffle = hyperparams['train_set_shuffle']
  train_batch_size = hyperparams["train_batch_size"]
  train_data_used_num = int(len(train_dataset) * hyperparams["train_data_used"])
  train_indices = torch.randperm(len(train_dataset))[:train_data_used_num]

  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=shuffle)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
  return train_loader, test_loader

In [13]:
def run_experiments(hyperparams, train_loader, test_loader, num_epochs, num_classes=NUM_CLASSES, in_channels=IN_CHANNELS):
    # Initialize model
    model = Inception(in_channels, num_classes).to(Training_Device)  # Assuming Inception is defined elsewhere

    # Use the optimizer from hyperparameters
    optimizer = hyperparams['optimizer'](model.parameters(), lr=hyperparams['initial_lr'])
    criterion = hyperparams['criterion']

    train_accuracy = {}
    test_accuracy = {}

    for epoch in range(num_epochs):
            # Assuming train and test functions are defined elsewhere
            train_acc = train(epoch, model, train_loader, optimizer, criterion, Training_Device)
            test_acc = test(epoch, model, test_loader, criterion, Training_Device)

            train_accuracy["train accuracy epoch"+str(epoch)] = train_acc
            test_accuracy["test accuracy epoch"+str(epoch)] = test_acc



    print("deeplift")
    dl_attributions = print_deeplift(test_loader, model, Feature_Attribution_Device)
    print("integrated_gradient")
    ig_attributions = print_ig(test_loader, model, Feature_Attribution_Device)
    return ig_attributions, dl_attributions, train_accuracy, test_accuracy

# Functions for saving attribution

In [14]:
def run_experiments_and_save(hyperparams_combinations, train_dataset, test_dataset, csv_file, num_run=NUM_RUN, num_epochs=NUM_EPOCHS):
    """
    Run experiments for each combination of hyperparameters, get feature layer attributions for DeepLift and Integrated Gradients,
    save the results to a CSV file after each run, and skip any combinations that have already been run.

    :param hyperparams_combinations: List of dictionaries with hyperparameter combinations.
    :param train_dataset: Training dataset.
    :param test_dataset: Test dataset.
    :param csv_file: Path to the CSV file for saving results.
    :param num_epochs: Number of epochs for training.
    """

    # Check if the CSV file exists and load existing data
    if os.path.exists(csv_file):
        existing_data = pd.read_csv(csv_file)
    else:
        existing_data = pd.DataFrame()

    for combo in hyperparams_combinations:
        print("\n\n")
        print("Combination:")
        print(combo)
        for i in range(num_run):  # For each run index
            print("runtime" + str(i))
            # Prepare data for checking if it's already processed
            combo_check = combo.copy()
            combo_check['run'] = i

            # Convert combo_check to string representation
            combo_check = {k: str(v) for k, v in combo_check.items()}



            if not existing_data.empty:
                # Filter existing_data to only the columns of interest
                filtered_data = existing_data[list(combo_check.keys())]
                # Convert filtered_data to string representation
                filtered_data = filtered_data.astype(str)
                # Check if a row with the same combination exists
                if any((filtered_data == pd.Series(combo_check)).all(axis=1)):
                    print("Combination already processed, skipping...")
                    continue  # Skip if combination is already processed

                # Set seed for reproducibility
            random.seed(i)
            np.random.seed(i)
            torch.manual_seed(i)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(i)

            train_loader, test_loader = get_data_loader(combo, train_dataset, test_dataset)

                # Run experiments and compute attributions
            ig_attributions, dl_attributions, train_accuracy, test_accuracy = run_experiments(hyperparams=combo, train_loader=train_loader, test_loader=test_loader, num_epochs=num_epochs)
            for method in ['deeplift', 'integrated_gradients']:
              if method == 'deeplift':
                  attr = dl_attributions, train_accuracy, test_accuracy
              elif method == 'integrated_gradients':
                  attr = ig_attributions, train_accuracy, test_accuracy

              combo_results = combo.copy()
              for d in attr:
                  combo_results.update(d)

              combo_results['method'] = method
              combo_results['run'] = i

              # Convert all values in combo_results to strings
              combo_results = {k: str(v) for k, v in combo_results.items()}

              # Append results to the existing data
              existing_data = existing_data.append(combo_results, ignore_index=True)

            # Save the data to CSV after each run
            existing_data.to_csv(csv_file, index=False)

# Experiments

In [15]:
hyperparams_choice_list = {
    'initial_lr': [0.001, 0.0007, 0.0003],
    'optimizer': [torch.optim.Adam], # Example optimizer
    'criterion': [torch.nn.CrossEntropyLoss()],
    'train_data_used': [1],
    'train_set_shuffle': [True],
    'train_batch_size': [64, 32, 16, 8]
}

combinations = generate_hyperparameter_combinations(hyperparams_choice_list)
for combo in combinations:
    print(combo)

{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 32}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 16}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 8}
{'initial_lr': 0.0007, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
{'initial_lr': 0.0007, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train

In [ ]:
run_experiments_and_save(combinations, train_dataset, test_dataset, csv_file=file_path)




Combination:
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
runtime0
Combination already processed, skipping...
runtime1
Combination already processed, skipping...
runtime2
Combination already processed, skipping...
runtime3
Combination already processed, skipping...
runtime4
Combination already processed, skipping...
runtime5
Combination already processed, skipping...
runtime6
Combination already processed, skipping...
runtime7
Combination already processed, skipping...
runtime8
Combination already processed, skipping...
runtime9
Combination already processed, skipping...
runtime10
Combination already processed, skipping...
runtime11
Combination already processed, skipping...
runtime12
Combination already processed, skipping...
runtime13
Combination already processed, skipping...
runtime14
Combination already processed, skipping...



Combination:
{'initial

<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.9752, Train Accuracy = 71.41%
Epoch 0: Test Loss = 0.9289, Test Accuracy = 75.87%
Epoch 1: Train Loss = 0.7363, Train Accuracy = 79.50%
Epoch 1: Test Loss = 0.9014, Test Accuracy = 76.62%
Epoch 2: Train Loss = 0.6792, Train Accuracy = 81.27%
Epoch 2: Test Loss = 0.9562, Test Accuracy = 76.52%
deeplift
Layer: branch1.0
Average Attribution: 92.46576538085938
Layer: branch2.0
Average Attribution: 24.461727905273438
Layer: branch3.0
Average Attribution: 37.384271240234376
Layer: branch4.0
Average Attribution: 19.792141723632813
integrated_gradient
Layer: branch1.0
Average Attribution: 92.43088363216614
Layer: branch2.0
Average Attribution: 24.43330079159166
Layer: branch3.0
Average Attribution: 37.36820612134274
Layer: branch4.0
Average Attribution: 19.792140628050973
runtime11


<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.9762, Train Accuracy = 71.32%
Epoch 0: Test Loss = 1.0634, Test Accuracy = 72.86%
Epoch 1: Train Loss = 0.7295, Train Accuracy = 79.78%
Epoch 1: Test Loss = 1.0155, Test Accuracy = 75.16%
Epoch 2: Train Loss = 0.6722, Train Accuracy = 81.45%
Epoch 2: Test Loss = 0.8126, Test Accuracy = 80.39%
deeplift
Layer: branch1.0
Average Attribution: 24.21336212158203
Layer: branch2.0
Average Attribution: 65.03258056640625
Layer: branch3.0
Average Attribution: 114.0541015625
Layer: branch4.0
Average Attribution: 20.602262878417967
integrated_gradient
Layer: branch1.0
Average Attribution: 24.14595779525419
Layer: branch2.0
Average Attribution: 65.03031312226372
Layer: branch3.0
Average Attribution: 114.11691241707494
Layer: branch4.0
Average Attribution: 20.602260900438708
runtime12


...

<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.9094, Train Accuracy = 72.86%
Epoch 0: Test Loss = 0.7964, Test Accuracy = 78.56%
Epoch 1: Train Loss = 0.6729, Train Accuracy = 80.99%
Epoch 1: Test Loss = 0.7386, Test Accuracy = 80.60%
Epoch 2: Train Loss = 0.6044, Train Accuracy = 83.04%
Epoch 2: Test Loss = 0.7815, Test Accuracy = 79.75%
deeplift
Layer: branch1.0
Average Attribution: -12.262351989746094
Layer: branch2.0
Average Attribution: -100.57298583984375
Layer: branch3.0
Average Attribution: -31.222314453125
Layer: branch4.0
Average Attribution: 9.926940155029296
integrated_gradient
Layer: branch1.0
Average Attribution: -12.296711812804686
Layer: branch2.0
Average Attribution: -100.57193074678383
Layer: branch3.0
Average Attribution: -31.170779064011025
Layer: branch4.0
Average Attribution: 9.926939854355608
runtime14


<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.9350, Train Accuracy = 72.09%
Epoch 0: Test Loss = 0.8247, Test Accuracy = 77.82%
Epoch 1: Train Loss = 0.6708, Train Accuracy = 81.25%
Epoch 1: Test Loss = 0.8307, Test Accuracy = 77.67%
Epoch 2: Train Loss = 0.6149, Train Accuracy = 82.77%
Epoch 2: Test Loss = 0.7449, Test Accuracy = 81.42%
deeplift
Layer: branch1.0
Average Attribution: 8.761665344238281
Layer: branch2.0
Average Attribution: -7.030668640136719
Layer: branch3.0
Average Attribution: -28.385150146484374
Layer: branch4.0
Average Attribution: 18.240313720703124
integrated_gradient
Layer: branch1.0
Average Attribution: 8.798785232900773
Layer: branch2.0
Average Attribution: -6.988464936924457
Layer: branch3.0
Average Attribution: -28.382323485126044
Layer: branch4.0
Average Attribution: 18.240318122081103


<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
